In [1]:
import pandas as pd
import numpy as np
ROWS_PER_FRAME= 543
import tensorflow as tf


def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [2]:
interpreter = tf.lite.Interpreter(model_path="model.tflite")
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")

In [3]:
BASE_DIR = 'data/asl-signs/'
train=pd.read_csv(f'{BASE_DIR}/train.csv')
pq_path = f'{BASE_DIR}/train_landmark_files/16069/695046.parquet'
xyz=load_relevant_data_subset(pq_path)
prediction=prediction_fn(inputs=xyz)

In [4]:
sign=pd.Series(prediction['outputs']).argmax()

train['sign_ord'] = train['sign'].astype('category').cat.codes
SIGN2ORD = train[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
ORD2SIGN = train[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()

ORD2SIGN[sign]


'mad'

In [5]:
train.query('sequence_id == 695046') 

,path,participant_id,sequence_id,sign,sign_ord
87142,train_landmark_files/16069/695046.parquet,16069,695046,mad,138
